Maggy enables you to train with Tensorflow distributed optimizers.
Using Maggy, you have to make minimal changes in train your model in a distributed fashion.

### 0. Spark Session

Make sure you have a running Spark Session/Context available.

On Hopsworks, just run your notebook to start the spark application.

### 1. Model definition

Let's define the model we want to train. The layers of the model have to be defined in the \_\_init__ function.

Do not instantiate the class, otherwise you won't be able to use Maggy.

In [ ]:
from tensorflow import keras 
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

# you can use keras.Sequential(), you just need to override it 
# on a custom class and define the layers in __init__()
class NeuralNetwork(Sequential):
    
    def __init__(self):
        super().__init__()
        self.add(Dense(10,input_shape=(4,),activation='tanh'))
        self.add(Dense(8,activation='tanh'))
        self.add(Dense(6,activation='tanh'))
        self.add(Dense(3,activation='softmax'))

model = NeuralNetwork

### 2. Dataset creation

You can create the dataset here and pass it to the TfDistributedConfig, or creating it in the training function.

You need to change the dataset path is correct.

In [2]:
dataset_path = "hdfs:///Projects/demo_ml_meb10000/TourData/iris/iris.csv"

import pandas as pd

dataset = pd.read_csv(dataset_path)

dataset

     sepal_length  sepal_width  petal_length  petal_width    variety
0             5.1          3.5           1.4          0.2     Setosa
1             4.9          3.0           1.4          0.2     Setosa
2             4.7          3.2           1.3          0.2     Setosa
3             4.6          3.1           1.5          0.2     Setosa
4             5.0          3.6           1.4          0.2     Setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  Virginica
146           6.3          2.5           5.0          1.9  Virginica
147           6.5          3.0           5.2          2.0  Virginica
148           6.2          3.4           5.4          2.3  Virginica
149           5.9          3.0           5.1          1.8  Virginica

[150 rows x 5 columns]

In [3]:
X = dataset.iloc[:,0:4].values
y = dataset.iloc[:,4].values

from sklearn.preprocessing import LabelEncoder

encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

Y = pd.get_dummies(y1).values


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0) 

### 3. Defining the training function

The programming model is that you wrap the code containing the model training inside a wrapper function. Inside that wrapper function provide all imports and parts that make up your experiment.

The function should return the metric that you want to optimize for. This should coincide with the metric being reported in the Keras callback (see next point).
You can return the metric list, in this case only the loss element will be printed.

In [ ]:
def training_function(model, train_set, test_set):
    model.build()
    #fitting the model and predicting

    model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])

    model.fit(train_set[0],train_set[1],epochs=20)

    accuracy = model.evaluate(
        x=test_set[0], y=test_set[1]
        )

    return accuracy

### 4. Configuring the experiment

In order to use maggy distributed training, we have to configure the training model, we can pass it to TfDistributedConfig.
the model class has to be an implementation of __tf.keras.Model__.
We can also define __train_set__, __test_set__ and eventually the __model_parameters__. __model_parameters__ is a dictionary
containing the parameters to be used in the \_\_init__ function of your model.

In [4]:
from maggy.experiment_config.tf_distributed import TfDistributedConfig

config = TfDistributedConfig(name="tf_test", module=model, train_set=(X_train, y_train), test_set=(X_test, y_test))




### 5. Run distributed training

Finally, we are ready to launch the maggy experiment. You just need to pass 2 parameters: the training function and the configuration variable we defined in the previous steps.

In [6]:
from maggy import experiment

experiment.lagom(training_function, config)

1: All executors registered: True
1: Epoch 1/20
1: 
1: 
0: All executors registered: True
0: Epoch 1/20
0: 
0: 
1: Epoch 2/20
1: Epoch 3/20
1: Epoch 4/20
1: Epoch 5/20
1: Epoch 6/20
0: Epoch 2/20
0: Epoch 3/20
0: Epoch 4/20
0: Epoch 5/20
0: Epoch 6/20
0: Epoch 7/20
0: Epoch 8/20
0: Epoch 9/20
0: Epoch 10/20
0: Epoch 11/20
0: Epoch 12/20
0: Epoch 13/20
0: Epoch 14/20
0: Epoch 15/20
0: Epoch 16/20
0: Epoch 17/20
0: Epoch 18/20
0: Epoch 19/20
0: Epoch 20/20
1: Epoch 7/20
1: Epoch 8/20
1: Epoch 9/20
1: Epoch 10/20
1: Epoch 11/20
1: Epoch 12/20
1: Epoch 13/20
1: Epoch 14/20
1: Epoch 15/20
1: Epoch 16/20
1: Epoch 17/20
1: Epoch 18/20
1: Epoch 19/20
1: Epoch 20/20
1: 
1: 
You are running Maggy on Hopsworks.
Final average test loss: 0.059
Finished experiment. Total run time: 0 hours, 0 minutes, 31 seconds
{'test result': 0.05948734097182751}
